In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, sys, re, gc

from utils import DatasetBuilder, maximize_window, assign_labels, build_path_features, get_children_features, build_parent_features
from time import sleep

2021-02-27 06:57:27,767 - INFO - dataset:dataset.py:16 - dataset package is loaded...


In [2]:
with DatasetBuilder(url='https://jdi-testing.github.io/jdi-light/') as builder:
    ds = builder.dataset
    display(ds)

2021-02-27 05:51:35,301 - WARNING - dataset:dataset.py:161 - The default dataset name "dummy" will be used
2021-02-27 05:51:35,302 - INFO - dataset:dataset.py:172 - Create directories to save the dataset
2021-02-27 05:51:35,303 - INFO - dataset:dataset.py:201 - Creating driver
2021-02-27 05:51:36,445 - INFO - dataset:dataset.py:208 - Chrome web driver is created
2021-02-27 05:51:36,446 - INFO - dataset:dataset.py:222 - getting url: https://jdi-testing.github.io/jdi-light/
2021-02-27 05:51:40,568 - INFO - common:common.py:82 - Window maximized
Number of discovered elements: 228


  0%|          | 0/228 [00:00<?, ?it/s]

2021-02-27 05:52:05,497 - INFO - dataset:dataset.py:231 - Save color screenshot to dataset/images/dummy.png
2021-02-27 05:52:05,790 - INFO - dataset:dataset.py:233 - Save html to dataset/html/dummy.html
2021-02-27 05:52:05,791 - INFO - dataset:dataset.py:130 - Build paths


  0%|          | 0/228 [00:00<?, ?it/s]

2021-02-27 05:52:06,680 - INFO - dataset:dataset.py:237 - Save parquet to dataset/df/dummy.parquet
2021-02-27 05:52:06,721 - INFO - dataset:dataset.py:249 - build_features method is not implemented


,parent_id,element_id,tag_name,x,y,height,width,displayed,enabled,selected,...,is_hover,base64png_before_hover,base64png_after_hover,attr_class,attr_onclick,attr_type,attr_role,attr_id,path,num_followers
0,None,e808d688-fcf8-4c2c-be5e-d116e2ead7cc,html,0,0,1319,1183,True,True,False,...,False,None,None,,None,None,None,,,227
1,e808d688-fcf8-4c2c-be5e-d116e2ead7cc,35340e3a-5688-4879-b1d4-ab54e162f77a,head,0,0,0,0,False,True,False,...,False,None,None,,None,None,None,,html:1183:1319,15
2,35340e3a-5688-4879-b1d4-ab54e162f77a,0139ea98-c80d-4636-a8f3-b29e51f1e356,meta,0,0,0,0,False,True,False,...,False,None,None,,None,None,None,,head:0:0/html:1183:1319,0
3,35340e3a-5688-4879-b1d4-ab54e162f77a,19493601-9b34-4f48-a758-1b017aebb062,title,0,0,0,0,False,True,False,...,False,None,None,,None,None,None,,head:0:0/html:1183:1319,0
4,35340e3a-5688-4879-b1d4-ab54e162f77a,a823a3da-fe4b-477d-8085-4dfe326c7b89,meta,0,0,0,0,False,True,False,...,False,None,None,,None,None,None,,head:0:0/html:1183:1319,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,988cb550-23f3-41cb-b26e-52ca1be02fc0,e303a49e-6fdf-424a-acd3-d1a36df9bab1,li,1098,1297,15,3,True,True,False,...,False,None,None,,None,,None,,ul:142:15/div:1200:64/div:1200:64/footer:1200:...,0
224,988cb550-23f3-41cb-b26e-52ca1be02fc0,2a3424cf-0e6b-429d-bada-84b1106a2868,li,1116,1297,15,64,True,True,False,...,False,None,None,,None,,None,,ul:142:15/div:1200:64/div:1200:64/footer:1200:...,1
225,2a3424cf-0e6b-429d-bada-84b1106a2868,6c512e9a-8373-4f44-95d3-bef1ebaef62e,a,1116,1297,15,64,True,True,False,...,False,None,None,,None,,None,,li:64:15/ul:142:15/div:1200:64/div:1200:64/foo...,0
226,028daa67-ac9a-4fb6-8ef6-57b189bf2f01,43b54172-7d01-410e-b21d-e03a36473f0f,script,0,0,18,223,False,True,False,...,False,None,None,,None,,None,,body:1200:1336/html:1183:1319,0


2021-02-27 05:52:06,743 - INFO - dataset:dataset.py:186 - Close web driver


In [3]:
SITE_ROOT = 'https://jdi-testing.github.io/jdi-light/'
LOGIN = 'Roman'
PASSWORD = 'Jdi1234'
SAVE_SCREEN = True
WAIT_TIME_SECONDS = 7


In [7]:
class JDI_DatasetBuilder(DatasetBuilder):
    
    def __init__(self, url = SITE_ROOT):
        super(JDI_DatasetBuilder, self).__init__(url)
        
    def setUp(self, driver):
        self.logger.info(f'getting page')
        driver.get(SITE_ROOT)
        
        driver.find_element_by_id("user-icon").click()
        driver.find_element_by_id("name").send_keys(LOGIN)
        driver.find_element_by_id("password").send_keys(PASSWORD)
        driver.find_element_by_id("login-button").click()
        sleep(WAIT_TIME_SECONDS)
        maximize_window(driver=driver)
        
        

In [8]:
with JDI_DatasetBuilder() as builder:
    pass

2021-02-27 01:38:31,336 - WARNING - dataset:dataset.py:136 - The default dataset name "dummy" will be used
2021-02-27 01:38:31,336 - INFO - dataset:dataset.py:141 - Construct
2021-02-27 01:38:31,337 - INFO - dataset:dataset.py:148 - Create directories to save the dataset
2021-02-27 01:38:31,338 - INFO - dataset:dataset.py:177 - Creating driver
2021-02-27 01:38:32,456 - INFO - dataset:dataset.py:184 - Chrome web driver is created
2021-02-27 01:38:32,457 - INFO - <ipython-input-7-52ef3e7186e5>:<ipython-input-7-52ef3e7186e5>:7 - getting page
2021-02-27 01:38:40,799 - INFO - common:common.py:82 - Window maximized
Number of discovered elements: 228


  0%|          | 0/228 [00:00<?, ?it/s]

2021-02-27 01:40:55,901 - INFO - dataset:dataset.py:111 - Build paths
2021-02-27 01:40:56,376 - INFO - dataset:dataset.py:208 - Save color screenshot to dataset/images/dummy.png
2021-02-27 01:40:56,668 - INFO - dataset:dataset.py:210 - Save parquet to dataset/df/dummy.parquet
2021-02-27 01:40:56,676 - INFO - dataset:dataset.py:212 - Save html to dataset/html/dummy.html
2021-02-27 01:40:56,682 - INFO - dataset:dataset.py:222 - build_features method is not implemented
2021-02-27 01:40:56,683 - INFO - dataset:dataset.py:162 - Close web driver


In [2]:
with DatasetBuilder(url = 'https://lenta.ru', dataset_name='lenta') as builder:
    print(builder.dataset.shape)

2021-02-27 02:15:27,091 - INFO - dataset:dataset.py:138 - Dataset name: lenta
2021-02-27 02:15:27,091 - INFO - dataset:dataset.py:141 - Construct
2021-02-27 02:15:27,093 - INFO - dataset:dataset.py:148 - Create directories to save the dataset
2021-02-27 02:15:27,093 - INFO - dataset:dataset.py:177 - Creating driver
2021-02-27 02:15:28,231 - INFO - dataset:dataset.py:184 - Chrome web driver is created
2021-02-27 02:15:28,232 - INFO - dataset:dataset.py:198 - getting url: https://lenta.ru
2021-02-27 02:15:33,721 - INFO - common:common.py:82 - Window maximized
Number of discovered elements: 2134


  0%|          | 0/2134 [00:00<?, ?it/s]

2021-02-27 02:20:24,945 - INFO - dataset:dataset.py:111 - Build paths
2021-02-27 02:20:31,979 - INFO - dataset:dataset.py:208 - Save color screenshot to dataset/images/lenta.png
2021-02-27 02:20:34,608 - INFO - dataset:dataset.py:210 - Save parquet to dataset/df/lenta.parquet
2021-02-27 02:20:34,727 - INFO - dataset:dataset.py:212 - Save html to dataset/html/lenta.html
2021-02-27 02:20:34,792 - INFO - dataset:dataset.py:222 - build_features method is not implemented
(2134, 20)
2021-02-27 02:20:34,793 - INFO - dataset:dataset.py:162 - Close web driver


In [3]:
with DatasetBuilder(url = 'https://www.cnews.ru/', dataset_name='cnews') as builder:
    print(builder.dataset.shape)

2021-02-27 02:22:12,376 - INFO - dataset:dataset.py:138 - Dataset name: cnews
2021-02-27 02:22:12,377 - INFO - dataset:dataset.py:141 - Construct
2021-02-27 02:22:12,378 - INFO - dataset:dataset.py:148 - Create directories to save the dataset
2021-02-27 02:22:12,379 - INFO - dataset:dataset.py:177 - Creating driver
2021-02-27 02:22:13,514 - INFO - dataset:dataset.py:184 - Chrome web driver is created
2021-02-27 02:22:13,515 - INFO - dataset:dataset.py:198 - getting url: https://www.cnews.ru/
2021-02-27 02:22:19,521 - INFO - common:common.py:82 - Window maximized
Number of discovered elements: 2194


  0%|          | 0/2194 [00:00<?, ?it/s]

2021-02-27 02:26:27,777 - INFO - dataset:dataset.py:111 - Build paths
2021-02-27 02:26:34,358 - INFO - dataset:dataset.py:208 - Save color screenshot to dataset/images/cnews.png
2021-02-27 02:26:35,626 - INFO - dataset:dataset.py:210 - Save parquet to dataset/df/cnews.parquet
2021-02-27 02:26:35,643 - INFO - dataset:dataset.py:212 - Save html to dataset/html/cnews.html
2021-02-27 02:26:35,671 - INFO - dataset:dataset.py:222 - build_features method is not implemented
(2194, 20)
2021-02-27 02:26:35,674 - INFO - dataset:dataset.py:162 - Close web driver


In [4]:
with DatasetBuilder(url = 'https://www.google.com/', dataset_name='google') as builder:
    print(builder.dataset.shape)

2021-02-27 02:26:40,161 - INFO - dataset:dataset.py:138 - Dataset name: google
2021-02-27 02:26:40,162 - INFO - dataset:dataset.py:141 - Construct
2021-02-27 02:26:40,163 - INFO - dataset:dataset.py:148 - Create directories to save the dataset
2021-02-27 02:26:40,164 - INFO - dataset:dataset.py:177 - Creating driver
2021-02-27 02:26:41,258 - INFO - dataset:dataset.py:184 - Chrome web driver is created
2021-02-27 02:26:41,259 - INFO - dataset:dataset.py:198 - getting url: https://www.google.com/
2021-02-27 02:26:44,932 - INFO - common:common.py:82 - Window maximized
Number of discovered elements: 197


  0%|          | 0/197 [00:00<?, ?it/s]

2021-02-27 02:27:00,987 - INFO - dataset:dataset.py:111 - Build paths
2021-02-27 02:27:01,426 - INFO - dataset:dataset.py:208 - Save color screenshot to dataset/images/google.png
2021-02-27 02:27:01,646 - INFO - dataset:dataset.py:210 - Save parquet to dataset/df/google.parquet
2021-02-27 02:27:01,651 - INFO - dataset:dataset.py:212 - Save html to dataset/html/google.html
2021-02-27 02:27:01,670 - INFO - dataset:dataset.py:222 - build_features method is not implemented
(197, 20)
2021-02-27 02:27:01,672 - INFO - dataset:dataset.py:162 - Close web driver


In [7]:
with DatasetBuilder(url = 'https://yandex.ru/covid19/stat', dataset_name='yandex-covid') as builder:
    print(builder.dataset.shape)

2021-02-27 02:39:05,330 - INFO - dataset:dataset.py:138 - Dataset name: yandex-covid
2021-02-27 02:39:05,331 - INFO - dataset:dataset.py:141 - Construct
2021-02-27 02:39:05,331 - INFO - dataset:dataset.py:148 - Create directories to save the dataset
2021-02-27 02:39:05,332 - INFO - dataset:dataset.py:177 - Creating driver
2021-02-27 02:39:06,449 - INFO - dataset:dataset.py:184 - Chrome web driver is created
2021-02-27 02:39:06,450 - INFO - dataset:dataset.py:198 - getting url: https://yandex.ru/covid19/stat
2021-02-27 02:39:10,790 - INFO - common:common.py:82 - Window maximized
Number of discovered elements: 4961


  0%|          | 0/4961 [00:00<?, ?it/s]

2021-02-27 02:55:51,514 - INFO - dataset:dataset.py:111 - Build paths
2021-02-27 02:56:16,394 - INFO - dataset:dataset.py:208 - Save color screenshot to dataset/images/yandex-covid.png
2021-02-27 02:56:17,190 - INFO - dataset:dataset.py:210 - Save parquet to dataset/df/yandex-covid.parquet
2021-02-27 02:56:17,212 - INFO - dataset:dataset.py:212 - Save html to dataset/html/yandex-covid.html
2021-02-27 02:56:17,281 - INFO - dataset:dataset.py:222 - build_features method is not implemented
(4961, 20)
2021-02-27 02:56:17,282 - INFO - dataset:dataset.py:162 - Close web driver


In [3]:
with DatasetBuilder(url = 'https://www.yandex.ru/', dataset_name='yandex') as builder:
    print(builder.dataset.shape)

2021-02-27 03:03:00,046 - INFO - dataset:dataset.py:138 - Dataset name: yandex
2021-02-27 03:03:00,047 - INFO - dataset:dataset.py:141 - Construct
2021-02-27 03:03:00,048 - INFO - dataset:dataset.py:148 - Create directories to save the dataset
2021-02-27 03:03:00,049 - INFO - dataset:dataset.py:177 - Creating driver
2021-02-27 03:03:01,178 - INFO - dataset:dataset.py:184 - Chrome web driver is created
2021-02-27 03:03:01,179 - INFO - dataset:dataset.py:198 - getting url: https://www.yandex.ru/
2021-02-27 03:03:05,713 - INFO - common:common.py:82 - Window maximized
Number of discovered elements: 3436


  0%|          | 0/3436 [00:00<?, ?it/s]

2021-02-27 03:15:31,114 - INFO - dataset:dataset.py:207 - Save color screenshot to dataset/images/yandex.png
2021-02-27 03:15:32,568 - INFO - dataset:dataset.py:209 - Save html to dataset/html/yandex.html
2021-02-27 03:15:32,572 - INFO - dataset:dataset.py:111 - Build paths
2021-02-27 03:15:54,612 - INFO - dataset:dataset.py:213 - Save parquet to dataset/df/yandex.parquet
2021-02-27 03:15:54,799 - INFO - dataset:dataset.py:225 - build_features method is not implemented
(3436, 20)
2021-02-27 03:15:54,800 - INFO - dataset:dataset.py:162 - Close web driver


<hr style="height: 5px">

In [15]:
df = pd.read_parquet('dataset/df/angular.parquet')
df = assign_labels(df, annotations_file_path='dataset/annotations/angular.txt', img=plt.imread('dataset/images/angular.png'))
df = build_path_features(elements_df=df)
df = build_parent_features(elements_df=df)
children_features = get_children_features(df)

Assigning labels:   0%|          | 0/522 [00:00<?, ?it/s]

  0%|          | 0/6447 [00:00<?, ?it/s]

2021-02-27 07:09:26,980 - INFO - dataset:dataset.py:126 - Building "parent_width"
2021-02-27 07:09:26,989 - INFO - dataset:dataset.py:130 - Building "parent_height"
2021-02-27 07:09:27,001 - INFO - dataset:dataset.py:134 - Building "parent_tag_name"
2021-02-27 07:09:27,014 - INFO - dataset:dataset.py:138 - Building "parent_is_hover"
2021-02-27 07:09:27,029 - INFO - dataset:dataset.py:142 - Building "parent_displayed"
2021-02-27 07:09:27,039 - INFO - dataset:dataset.py:44 - select all leafs (nodes which are not parents)
2021-02-27 07:09:27,041 - INFO - dataset:dataset.py:46 - Leafs set size: 3249 (nodes which have no children)
2021-02-27 07:09:27,046 - INFO - dataset:dataset.py:49 - count number of references to leafs
2021-02-27 07:09:27,050 - INFO - dataset:dataset.py:51 - Nodes with leafs as children set size: 1749 (nodes which have leafs as children)
2021-02-27 07:09:27,054 - INFO - dataset:dataset.py:54 - count num children for each node
2021-02-27 07:09:27,055 - INFO - dataset:data

In [16]:
# df['is_leaf'] = df.element_id.apply(lambda x: 1 if x in children_features['leafs'] else 0)

In [17]:
# df['num_leafs'] = df.element_id.map(children_features['num_leafs'])

In [18]:
# df['num_children'] = df.element_id.map(children_features['num_children'])

In [19]:
columns = ['tag_name',
           'x',
           'y',
           'parent_tag_name', 
           'parent_width', 
           'parent_height', 
           'width', 
           'height', 
           'num_leafs', 
           'is_leaf', 
           'num_followers', 
           'num_children', 
           'parent_is_hover',
           'is_hover',
           'parent_displayed',
           'displayed',
           'sum_children_widths',
           'sum_children_hights',
           'label',
          ]

In [20]:
df[columns]

,tag_name,x,y,parent_tag_name,parent_width,parent_height,width,height,num_leafs,is_leaf,num_followers,num_children,parent_is_hover,is_hover,parent_displayed,displayed,sum_children_widths,sum_children_hights,label
0,html,0,0,html,0.0,0.0,1183.0,44785.0,0,0,6446,2,0,1,0,1,1200.0,44802.0,-1.0
1,head,0,0,html,1183.0,44785.0,0.0,0.0,169,0,169,169,1,0,1,0,0.0,0.0,-1.0
2,meta,0,0,head,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0.0,0.0,-1.0
3,title,0,0,head,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0.0,0.0,-1.0
4,meta,0,0,head,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0.0,0.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6442,li,1092,44763,ul,149.0,14.0,3.0,14.0,0,1,0,0,0,0,1,1,0.0,0.0,-1.0
6443,li,1111,44763,ul,149.0,14.0,70.0,14.0,1,0,1,1,0,1,1,1,69.0,14.0,1.0
6444,a,1111,44763,li,70.0,14.0,69.0,14.0,0,1,0,0,1,1,1,1,0.0,0.0,-1.0
6445,script,0,16,body,1200.0,44802.0,234.0,16.0,0,1,0,0,1,0,1,0,0.0,0.0,-1.0


In [22]:
df[df.label>=0][columns].sort_values(by='label', ascending=False).head(30)

,tag_name,x,y,parent_tag_name,parent_width,parent_height,width,height,num_leafs,is_leaf,num_followers,num_children,parent_is_hover,is_hover,parent_displayed,displayed,sum_children_widths,sum_children_hights,label
5447,table,230,39128,table-footer-row-example,940.0,444.000,940.0,393.0,0,0,27,3,0,0,1,1,2820.0,392.0,19.0
5054,table,230,36306,sort-overview-example,940.0,163.000,940.0,96.0,0,0,76,6,0,0,1,1,5640.0,96.0,19.0
5902,table,230,41710,table-row-context-example,940.0,348.000,940.0,297.0,1,0,51,3,0,0,1,1,1880.0,296.0,19.0
5719,div,230,40825,table-overview-example,940.0,388.375,940.0,267.0,0,0,92,2,0,0,1,1,1880.0,267.0,19.0
5958,table,230,42134,table-selection-example,940.0,348.000,940.0,297.0,1,0,117,3,0,0,1,1,1880.0,296.0,19.0
5478,div,230,39648,table-http-example,940.0,407.000,940.0,356.0,0,0,188,2,0,0,1,1,1880.0,356.0,19.0
5137,table,230,36577,table-basic-example,940.0,348.000,940.0,297.0,1,0,33,3,0,0,1,1,1880.0,296.0,19.0
5399,table,230,38608,table-filtering-example,940.0,514.125,940.0,393.0,1,0,43,3,0,0,1,1,1880.0,392.0,19.0
5670,table,230,40131,table-multiple-header-footer-example,940.0,548.000,940.0,497.0,0,0,32,3,0,0,1,1,2820.0,496.0,19.0
6080,table,230,42558,table-sorting-example,940.0,444.000,940.0,393.0,1,0,75,3,0,0,1,1,1880.0,392.0,19.0


In [10]:
df[df.is_hover==1][columns]

,tag_name,parent_tag_name,parent_width,parent_height,width,height,num_leafs,is_leaf,num_followers,num_children,parent_is_hover,is_hover,parent_displayed,displayed,sum_children_widths,sum_children_hights,label
0,html,html,0.0,0.0,1183.0,44785.0,0,0,6446,2,0,1,0,1,1200.0,44802.0,-1.0
171,body,html,1183.0,44785.0,1200.0,44802.0,3,0,6275,7,1,1,1,1,4298.0,44931.0,-1.0
172,header,body,1200.0,44802.0,1200.0,60.0,0,0,72,1,1,1,1,1,1200.0,60.0,-1.0
173,div,header,1200.0,60.0,1200.0,60.0,0,0,71,1,1,1,1,1,1200.0,0.0,-1.0
183,ul,nav,1200.0,0.0,420.0,60.0,0,0,28,4,0,1,1,1,420.0,240.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6320,tr,tbody,923.0,480.0,923.0,48.0,4,0,4,4,0,1,1,1,923.0,192.0,-1.0
6440,li,ul,149.0,14.0,31.0,14.0,1,0,1,1,0,1,1,1,31.0,14.0,1.0
6441,a,li,31.0,14.0,31.0,14.0,0,1,0,0,1,1,1,1,0.0,0.0,-1.0
6443,li,ul,149.0,14.0,70.0,14.0,1,0,1,1,0,1,1,1,69.0,14.0,1.0
